# Quiz-4

Goals:

* Learn about NFA
* Learn Brzozowski's minimization algorithm for DFA
  - Involves DFA reversal which generates NFA
* Learn about regular expressions and relationship with NFA
* Become proficient with these topics, especially in light of Asg-2 and MT-1

Submission:

* Clear demonstration that you ran this notebook and made observations

* Please make your observations by adding markdown cells

* Points will be based on a significant number of observations pertaining to each new algorithm or idea

  - No need to add observations about concepts and ideas that are illustrated on a second or third example. Just do it for the first occurrence.

In [ ]:
# PLAY this Youtube to know how to use the contents of this file + follow some of the Def_DFA.ipynb defns
# The notebook that is mentioned in this Youtube video may be found
# on the Git repo

from IPython.display import YouTubeVideo
YouTubeVideo('xjFtLF95uBc')

In [ ]:
from jove.DotBashers import *
from jove.Def_md2mc import *
from jove.Def_NFA   import *
from jove.Def_DFA   import *
from jove.Def_RE2NFA    import *
from jove.Def_NFA2RE    import *

An NFA is a machine $(Q, \Sigma, \delta, Q_0, F)$ which is somewhat like a DFA 
except that 

1. It can start from a __set__ of starting states $Q_0$
    > i.e., the NFA can start from more than one starting state
    
2. Its transition function $\delta$ maps $Q\times (\Sigma\cup \{\varepsilon\})$ to $2^{Q}$
    > i.e., the NFA takes a state $Q$ and a symbol and returns a set of states
    
You can see these aspects being illustrated in the NFA to follow

# Limitations of DFA

In [ ]:
secondlast = md2mc('''
NFA
I  : 0 -> I
I  : 1 -> I, S0
S0 : 0 | 1 -> F
''')
dotObj_nfa(secondlast, FuseEdges=True)

In [ ]:
thirdlast = md2mc('''
NFA
I  : 0 -> I
I  : 1 -> I, S0
S0 : 0 | 1 -> S1
S1 : 0 | 1 -> F
''')
dotObj_nfa(thirdlast, FuseEdges=True)

In [ ]:
fourthlast = md2mc('''
NFA
I  : 0 -> I
I  : 1 -> I, S0
S0 : 0 | 1 -> S1
S1 : 0 | 1 -> S2
S2 : 0 | 1 -> F
''')
dotObj_nfa(fourthlast, FuseEdges=True)

In [ ]:
fifthlast = md2mc('''
NFA
I  : 0 -> I
I  : 1 -> I, S0
S0 : 0 | 1 -> S1
S1 : 0 | 1 -> S2
S2 : 0 | 1 -> S3
S3 : 0 | 1 -> F
''')
dotObj_nfa(fifthlast, FuseEdges=True)

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(secondlast)))

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(thirdlast)))

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(fourthlast)))

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(fifthlast)))

In [ ]:
len(min_dfa(nfa2dfa(secondlast))["Q"])

In [ ]:
len(min_dfa(nfa2dfa(thirdlast))["Q"])

In [ ]:
len(min_dfa(nfa2dfa(fourthlast))["Q"])

In [ ]:
len(min_dfa(nfa2dfa(fifthlast))["Q"])

## Clear evidence of exponential blowup!

### Another problem with DFA : No natural way to specify many languages

In [ ]:
# NFA for the language {'a','ba','cd','eb'}
nfa_abcde = md2mc('''
NFA
I   : a -> F
I   : b -> Sb
Sb  : a -> F
I   : c -> Sc
Sc  : d -> F
I   : e -> Se
Se  : b -> F
''')
dotObj_nfa(nfa_abcde)

In [ ]:
# With DFA we do something else (build DFA for 'a' with alphabet being {a,b,c,d,e} etc...)
# This results in this DFA
dotObj_dfa_w_bh(min_dfa(nfa2dfa(nfa_abcde)), FuseEdges=True)

# Chapter-7: Nondeterministic Finite Automata

In this chapter, we will cover virtually all aspects of NFA, following the style of presentation used in Chapter-3 on DFA. We will sometimes be (re-) writing print (and dot-object generation) routines that look quite similar to those defined for DFA. This is because the routines are short, and we want a self-contained notebook. Besides there are subtle differences between an NFA and a DFA and it's best to make these differences manifest in dedicated routines -- than overload the former routines with extra arguments. 

<span style="color:blue"> **We begin defining NFA by providing its structure. ** </span>

We will first set up a series of definitions ending at the mk_nfa function that helps build NFA. 

Unlike with a DFA, there is no mkp_nfa function, as all NFAs are partial! (We can define a mktot_nfa function to print all moves, including unspecified moves leading to the se({}) state.)

We will define step_nfa, run_nfa and accepts_nfa which are key functions that clearly spell out how NFA differ from DFA.

We will also be defining functions for displaying NFA with the help of the _dot_ tool. The design will be similar to that with DFA.

<span style="color:blue"> **------** </span>

<span style="color:red"> __We will follow Kozen and endow an NFA with multiple start states __ </span>



This will allow the NFA to be more naturally handled. For instance, the reverse of a DFA is an NFA. When we reverse a DFA, all its final states become initial states of the NFA (that models the reversed language). There are 2 ways to handle this:

1. Introduce a fake new initial state and jump from it via $\varepsilon$ onto (what were the final state of the DFA).

2. Don't introduce the fake new initial state, but rather allow the NFA to start from all of F being really its start state.

    * Of course, in almost all situations, this is a minor difference
    
    * But to enjoy the topic as completely as one can, it is best to be "clean" and follow clean definitions.
    
        - I've seen Ed Clarke also use multiple initial states
        
        - Hence our code will be for this relaxed setup
        
        - Of course if you __REALLY__ wanted to have only one start state, then make it a singleton set of states and work that through your NFA.


<br>

__So now, following Brzozowski, we have__

<br>

An NFA is a quintuple $(Q,\Sigma,\delta,Q_0,F)$, where:

* $Q$ is a _finite nonempty_ set of states.

* $\Sigma$ is a _finite nonempty_ alphabet containing _symbols_.

* $\delta$ is a (partial)
	transition function, containing a set of _transitions_. The transitions take
    a pair from $Q\times \Sigma$ and return a __subset__ of states in $Q$. All this is succinctly
    captured by writing
    $\delta: Q\times \Sigma \rightarrow 2^Q$. 
    Here we use $2^Q$ to denote the powerset of $Q$.
    
  
* $Q_0\subseteq Q$, is __a set of initial states__.  Notice that we change from q0 (or $q_0$) which is what you find books such as Sipser and Linz using.


* $F\subseteq Q$ is a _finite_ (and _possibly empty_) set of
	final (or _accepting_) states. These are shown as double-circled nodes in the graph of a DFA. 
 
> There is no other change. I.e. $\delta$ remains the same as before.
> It is that when an NFA starts, it can find itself in a set of start states.
> Most NFAs start from a __singleton__ Q0, which is then, effectively, an NFA
 that matches most books say.

Some terminology:

> We call $Q$,$\Sigma$, $\delta$, $Q_0$, and $F$ the **_traits_** of the NFA.

> We will call an NFA **_structurally consistent_** or simply **"consistent"** if its traits pass the aforesaid checks.


Here is how the checks will be broken down:

* The supplied $\delta$ function will be checked to see if it has allowed domain and range points. 
 - The domain points must be a subset of $Q\times \Sigma$
 - The range points must be a subset of $2^Q$
  We do no insist that the supplied $\delta$ be total.
    
* $Q_0\subseteq Q$, is _the_ initial state.

* $F\subseteq Q$ is a _finite_ (and _possibly empty_) set of
	final (or _accepting_) states.  
    
We will often use the state set({}) to be the equivalent of a black-hole state for an NFA.

In [ ]:
nfa1 = md2mc('''NFA
I : 0 -> A
I : 0 -> F''')
dotObj_nfa(nfa1)

In [ ]:
nfa2 = md2mc('''NFA
I : '' -> F
I : 0  -> A
''')
dotObj_nfa(nfa2)

# The $\delta$ function of the NFA

In [ ]:
help(step_nfa)

In [ ]:
step_nfa(nfa1, 'I', '')

In [ ]:
step_nfa(nfa1, 'I', '0')

In [ ]:
step_nfa(nfa2, 'I','')

In [ ]:
fig71a = md2mc('''
NFA
I  : 0 -> I
I  : 1 -> I, S0
S0 : 0 | 1 -> S1
S1 : 0 | 1 -> F
''')

In [ ]:
dotObj_nfa(fig71a, FuseEdges=True)

In [ ]:
fig71b = md2mc('''
NFA
I  : 0 | 1 -> I
I  : '' -> S0
S0 : 1  -> S1
S1 : 0 | 1 -> S2
S2 : 0 | 1 -> F
''')

In [ ]:
dotObj_nfa(fig71b, FuseEdges=True)

In [ ]:
help(Eclosure)

In [ ]:
Eclosure(fig71b, {'F'})

# Stepping and Running NFA

Now that we've defined NFA and allied actions such as consistency checking and printing, let's write functions to step and run them.

* How the state transition function $\delta$ "works"
  - captured in step_nfa

In [ ]:
help(step_nfa)

In [ ]:
step_nfa(fig71b, "I", '')

In [ ]:
step_nfa(fig71b, "S0", '')

In [ ]:
step_nfa(fig71b, "I", '0')

In [ ]:
help(run_nfa)

In [ ]:
run_nfa(fig71b, "I", "0")

In [ ]:
run_nfa(fig71b, "I", "0", chatty=True)

In [ ]:
step_nfa(fig71b, "I", '1')

In [ ]:
run_nfa(fig71a, "I", '0100100', chatty = True)

In [ ]:
run_nfa(fig71b, "I", '1')

In [ ]:
run_nfa(fig71b, {"I"}, "")

In [ ]:
Eclosure(fig71b, {"I"})

# The EClosure Function (defined on a set of states)

In [ ]:
run_nfa(fig71b, {"I"}, "0101")

In [ ]:
run_nfa(fig71b, {"I"}, "0101", True)

In [ ]:
import ipywidgets as wdg
def run_nfa_slider(firstTime, N, s, n):
    """Run NFA N from N["Q0"] (which is a set..) on substring s[0:n]
    """
    if firstTime:
        print("Eclosure of N's start state is")
        print(Eclosure(N, N["Q0"]))
        firstTime = False
    S = N["Q0"] 
    if (n > len(s)):
        n = len(s)
    print("string = ", s[0:n])
    run_nfa(N, S, s[0:n], True)
    
def run_nfa_int(N1, N2):
    """Run interactively from the given NFA .. from {q0} 
       on input string's substring
       as picked by slider.
    """
    inp = input("Please provide string: ")
    wdg.interact(run_nfa_slider, firstTime=True, N = {'N1': N1, 'N2': N2}, 
                 s = inp, n=(0,32) )

In [ ]:
run_nfa_int(fig71a, fig71b)

In [ ]:
fig74a = md2mc('''
NFA
I : '' -> A, G
A : '' -> B, C
B : 1  -> D
C : 0  -> E
D : '' -> A, G
E : '' -> A, G
G : 1  -> F
''')
dotObj_nfa(fig74a)

In [ ]:
run_nfa_int(fig74a, fig71b)

# DFA to NFA conversion (!)

This is a useful helper and helps understand the theory, but not widely used.
Its main use is within "NFA2RE". Suppose you want to convert a DFA to an RE? 
Then use dfa2nfa and then NFA2RE :-)

In [ ]:
def dfa2nfa(D):
    """Given a DFA D, make a language-equivalent NFA.
    """
    assert(
    is_partially_consistent_dfa(D)
    ), "DFA given to dfa2nfa is not part. consist."
    return { "Q"     : D["Q"],
             "Sigma" : D["Sigma"],
             "Delta" : dict((a,{b}) for (a,b) in D["Delta"].items()),
             "Q0"    : { D["q0"] },
             "F"     : D["F"] }  

# NFA to DFA conversion


* Input: An NFA, N

* Output: A language-equivalent DFA, D

* Method: Subset Construction

  - Data structure to maintain : 
  
    * A set called "Unexpanded" (UNEXP for short), which holds SETS of states of the given NFA, N
    
    * These serve as the states of the DFA (D) being built
    
      - Note that UNEXP is a set of state-sets (set of 'set-of-states' if you will)

> * Let INIT  (DFA's initial state) =  Eclosure of N["Q0"]  

> * Add INIT to UNEXP

> WHILE (UNEXP $\neq \emptyset$) DO

> > Choose a state S from UNEXP

> > Delete S from UNEXP

> > Expand(S)   --   Expand(S) will add all the 'c' moves out of S where c $\in \Sigma$

> END WHILE 

** Expand(S): **

> For each symbol $c$ in $\Sigma$

> > For each state s ∈ S do  --  Recall that S is a set of states

> > > Let $NS_c$ = $\delta(s,c)$ --  Find the next __set of states__ the NFA can be, starting from s, moving on c 

> > > Let $NSE_c$ = Eclose($NS_c$)   --  Eclose $NS_c$ which means Eclose every state in $NS_c$ and union them

> > > Introduce a transition in D from S to $NSE_c$

> > If $NSE_c$ does not exist in the DFA D, add it to UNEXP

** Eclose(S): **

> For each state $x \in S$:

> > Move $x$ through $\varepsilon$, obtaining a set of next state $S_x$

> Union these $S_x$ and return that union

# The Actual NFA 2 DFA Conversion Code

In [ ]:
def nfa2dfa(N):
    """In : N (consistent NFA)
       Out: A consistent DFA that is language-equivalent to N.
    """
    assert(
        is_consistent_nfa(N)
    ), "nfa2dfa was given an inconsistent NFA."
    # EClose the starting state of the NFA
    EC = Eclosure(N, N["Q0"])
    return n2d(Frontier=[EC], Visited=[EC], Delta=dict({}), Nfa=N)


def n2d(Frontier, Visited, Delta, Nfa):
    """Helper for nfa2dfa.
       ---
       In : Frontier (list of state sets; initially Eclosed Q0)
            Visited  (list of visited state sets; initially Eclosed Q0)
            Delta    (the DFA transition function being formed)
            Nfa      (the NFA being converted)
       Helper to nfa2dfa. Given a (BFS) frontier, a Visited
       set of states, the Delta being formed, and NFA Nfa, see
       if all new moves are in Visited:
         do last gasp of Delta update; make and return a DFA;
       else: extend Frontier, Visited, Delta; recurse.
    """
    All_c_Moves = [ ((Q,c),ec_step_nfa(Nfa,Q,c)) 
                   for Q in Frontier 
                   for c in Nfa["Sigma"] ]
    New_c_Moves = list(filter(lambda QcQ: trTrg(QcQ) not in Visited, 
                              All_c_Moves))  
    if New_c_Moves == []:
        # Add last-gasp c-moves that curl back!
        last_gasp_c_moves = dict([ ((mkSSnam(Qfrom),c),mkSSnam(Qto)) 
                                  for ((Qfrom, c), Qto) in All_c_Moves ])
        Delta.update(last_gasp_c_moves)
                  
        # DFA states are visited states
        DFA_Q = { mkSSnam(Q) for Q in Visited }
                  
        # Retain alphabet
        DFA_Sigma = Nfa["Sigma"]
                  
        # Delta is ready to go
        DFA_Delta = Delta
                  
        # DFA starts at Eclosure of Nfa's Q0 set of states
        DFA_q0 = mkSSnam(Eclosure(Nfa, Nfa["Q0"]))
                  
        # DFA's final states are those in visited that contain an NFA 
        # F-state but don't retain any empty sets, in case the NFA given 
        # has no F-states!
        # This is another corner-case (i.e. don't shove-in black hole 
        # states!)
        DFA_F = set(map(lambda Q: mkSSnam(Q), 
                        filter(lambda Q: (Nfa["F"]&Q) != set({}), 
                               Visited)))
                  
        # Make the DFA; send it to the DFA-shrink to bask ugly long 
        # state names...
        return shrink_dfastates(mk_dfa(DFA_Q, 
                                       DFA_Sigma, 
                                       DFA_Delta, 
                                       DFA_q0, 
                                       DFA_F))
    else:
        newFrontier = list(map(lambda QcQ: trTrg(QcQ), New_c_Moves)) 
        newVisited = Visited + newFrontier
                  
        # Even though the NFA has not closed back on itself, we MUST 
        # accommodate for the "curl-backs" along the way !!  Thus, run it
        # over All_c_Moves which may include "partial revisits along the 
        # way". We MUST pick up those curl-backs!
        NewMovesDelta = dict([ ((mkSSnam(Qfrom),c),mkSSnam(Qto)) 
                              for ((Qfrom, c), Qto) in All_c_Moves ]) 
        Delta.update(NewMovesDelta)
        return n2d(newFrontier, newVisited, Delta, Nfa)
                                  
#---NFA to DFA

In [ ]:
fig74a = md2mc('''
NFA
I : '' -> A, G
A : '' -> B, C
B : 1  -> D
C : 0  -> E
D : '' -> A, G
E : '' -> A, G
G : 1  -> F
''')
dotObj_nfa(fig74a)

In [ ]:
dotObj_dfa(nfa2dfa(fig74a))

# Brzozowski's DFA Minimization

Picking up from our earlier discussions, to minimize a DFA using Brzozowski's algorithm, here are the steps:

* Make sure that the given DFA has no unreachable states
* Reverse the DFA
* Determinize it
* Reverse that DFA
* Determinize it

Thus we need to write a routine to reverse a DFA. We already have a way to ensure that a DFA does not have unreachable states (in another Jupyter notebook; we won't bother to include it here, and trust the user to always provide such DFA only).

We can observe that if a DFA has black-hole states, then those states won't matter in the reversed machine (reversed NFA). Thus, we can work with __partial__ dfa (i.e., DFA that are partially consistent).

## DFA reversal

In [ ]:
def inSets(D,trg,ch):
    """In : D   = partially consistent dfa,
            trg = a target state in D["q"]
            ch  = a member of D["Sigma"]
       Out: a set of states. { q s.t. Delta[q,ch] == trg }
    """
    return { q for q in D["Q"] if D["Delta"][(q,ch)] == trg }

def rev_dfa(D):
    """In : D = a partially consistent DFA without any unreachable states.
       Out: A consistent NFA whose language is D's language reversed.
    """
    # 1. Given that NFAs start from a SET of states, we already have that
    #   info. No need to add any transitions from "a new initial state" 
    #   etc
    
    # 2. Now add the inSets of each state as the NFA next set of states
    NDict = { (q,ch) : inSets(D,q,ch) 
              for q in D["Q"] 
              for ch in D["Sigma"] }
    
    # Notice that we retain D["Q"] and start from Q0 = D["F"]
    # going backwards along NDict toward F_dfa = { D["q0"] }
    return mk_nfa(D["Q"], D["Sigma"], NDict, D["F"], {D["q0"]})

In [ ]:
nfaMultiQ0 = md2mc('''
NFA
I0 : a | b | c -> A, B
I0 : c -> F
I1 : a | b -> A, B
A  : c -> F
B  : d -> F
''')

In [ ]:
dotObj_nfa(nfaMultiQ0)

In [ ]:
dotObj_nfa(nfaMultiQ0, FuseEdges=True)

In [ ]:
dfaMQ0 = nfa2dfa(nfaMultiQ0)

In [ ]:
dotObj_dfa(dfaMQ0)

In [ ]:
dotObj_dfa(dfaMQ0, FuseEdges=True)

In [ ]:
dotObj_nfa(rev_dfa(dfaMQ0))

In [ ]:
dotObj_nfa(rev_dfa(dfaMQ0), FuseEdges=True)

In [ ]:
help(min_dfa_brz)

In [ ]:
dotObj_dfa(dfaMQ0)

In [ ]:
dotObj_dfa(min_dfa_brz(dfaMQ0))

# Brzozowski Minimization : All Steps 

In [ ]:
blimp = md2mc('''
DFA 
I1 : a -> F2
I1 : b -> F3
F2 : a -> S8
F2 : b -> S5
F3 : a -> S7
F3 : b -> S4
S4 : a | b -> F6
S5 : a | b -> F6
F6 : a | b -> F6
S7 : a | b -> F6
S8 : a -> F6
S8 : b -> F9
F9 : a -> F9
F9 : b -> F6
''')

In [ ]:
DOblimp = dotObj_dfa(blimp)

In [ ]:
DOblimp

In [ ]:
dotObj_dfa(blimp, FuseEdges=True)

# Classical minimization results in the following

In [ ]:
classic_min = min_dfa(blimp)
dotObj_dfa(classic_min)

# Now let's study Brzozowski minimization; its code is a one-liner!

In [ ]:
# Brzozowski Minimizer code
def min_dfa_brz(D):
    """Minimize a DFA as per Brzozowski's algorithm.
    """
    return nfa2dfa(rev_dfa(nfa2dfa(rev_dfa(D))))

## Step 1: Reverse the given DFA

In [ ]:
rblimp = rev_dfa(blimp)
DOrblimp = dotObj_nfa(rblimp)
DOrblimp

In [ ]:
dotObj_nfa(rblimp, FuseEdges=True)

## Step 2: Determinize the result of Step 1

In [ ]:
drblimp = nfa2dfa(rblimp)
drblimp

In [ ]:
DOdrblimp = dotObj_dfa(drblimp)
DOdrblimp

## Step 3: Reverse the result of Step 2

In [ ]:
rdrblimp = rev_dfa(drblimp)
DOrdrblimp = dotObj_nfa(rdrblimp)
DOrdrblimp

## Step 4: Determinize the result of Step 3

In [ ]:
drdrblimp = nfa2dfa(rdrblimp)
DOdrdrblimp = dotObj_dfa(drdrblimp)
DOdrdrblimp

## End result is isomorphic!

In [ ]:
iso_dfa(drdrblimp, classic_min)

# RE to NFA

In [ ]:
dotObj_nfa(re2nfa("''"))

In [ ]:
dotObj_nfa(re2nfa("a"))

In [ ]:
dotObj_nfa(re2nfa("ab"))

In [ ]:
dotObj_nfa(re2nfa("a+b"))

In [ ]:
dotObj_nfa(re2nfa("a*"))

In [ ]:
dotObj_nfa(re2nfa("''*"))

In [ ]:
nfromr = re2nfa("ab(a+b)*")

In [ ]:
dotObj_nfa(nfromr)

In [ ]:
mk_gnfa

In [ ]:
help(mk_gnfa)

In [ ]:
gnfromr = mk_gnfa(nfromr)
dotObj_gnfa(gnfromr)

In [ ]:
del_gnfa_states

In [ ]:
help(del_gnfa_states)

In [ ]:
(Gf, DO, RE) = del_gnfa_states(gnfromr)

In [ ]:
RE

In [ ]:
DO[0]

In [ ]:
DO[6]

In [ ]:
dotObj_nfa(re2nfa("''"))

In [ ]:
dotObj_nfa(re2nfa("a"))

In [ ]:
dotObj_nfa(re2nfa('ab'))

In [ ]:
dotObj_nfa(re2nfa("a*"))

$\varepsilon$

In [ ]:
dotObj_nfa(re2nfa("a(a+b)*b"))

In [ ]:
dotObj_nfa(re2nfa("dotObj_nfa(re2nfa("''"))a+b"))

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(re2nfa("(0+1)*1(0+1)(0+1)"))))

In [ ]:
dotObj_dfa(min_dfa(nfa2dfa(re2nfa("(0+1)*1(0+1)(0+1)(0+1)(0+1)(0+1)"))))

In [ ]:
len(min_dfa(nfa2dfa(re2nfa("(0+1)*1(0+1)(0+1)(0+1)(0+1)(0+1)(0+1)")))["Q"])

In [ ]:
dotObj_nfa(re2nfa("''"))

# Split the cells below into separate cells and run

This way, you can observe the results after each command

In [ ]:
dotObj_nfa(re2nfa("a"))

dotObj_nfa(re2nfa("  (10)* (11)  "))

re2nfa("  (10)* (11)  ")

NFA1 = re2nfa("   (a+b+c)(p+q)*(m+n) " )

dotObj_nfa(mk_gnfa(NFA1))

GNFA1 = mk_gnfa(NFA1)

(Gnfa_fina, DO, REfinal) = del_gnfa_states(GNFA1)

REfinal

DO[0]

DO[1]

DO[2]

DO[5]

len(DO)

DO[13]

DO[15]

dotObj_dfa(comp_dfa(min_dfa(nfa2dfa(re2nfa(" (1+0)*(111+000+100+010+001)(1+0)* ")))))

In [ ]:
dotObj_nfa(re2nfa("''"))

dotObj_nfa(re2nfa("a"))

dotObj_nfa(re2nfa("ab"))

dotObj_nfa(re2nfa("a+b"))

dotObj_nfa(re2nfa("a*"))

dotObj_nfa(re2nfa("''*"))

nfromr = re2nfa("ab(a+b)*")

dotObj_nfa(nfromr)

dotObj_nfa(re2nfa("(01)*1+0"))

In [ ]:
mk_gnfa

help(mk_gnfa)

gnfromr = mk_gnfa(nfromr)
dotObj_gnfa(gnfromr)

del_gnfa_states

help(del_gnfa_states)

(Gf, DO, RE) = del_gnfa_states(gnfromr)

RE

DO[0]

DO[6]



DO[7]

DO[8]

DO[9]

dotObj_dfa(min_dfa(nfa2dfa(re2nfa(RE))))

# Review of Jove's capabilities

Jove helps teach models of computation using Jupyter 

Included are modules on:

* Sets, strings and languages
* Language operations
* Construction of and operations on DFA and NFA
* Regular expression parsing and automata inter-conversion
* Derivate-based parsing
* Pushdown automata
* The construction of parsers using context-free productions, including
  a full lexer/parser for Jove's own markdown syntax
* Studies of parsing: ambiguity, associativity, precedence
* Turing machines (including one for the Collatz problem)

For a complete Jove top-level reference, kindly refer to https://github.com/ganeshutah/Jove from where you can download and obtain Jove. You can also visit this Github link now and poke around (the NBViewer will display the contents).

Once you are in the top-level Gallery link we provide, feel free to explore the hierarchy of modules found there.

These notebooks should give you an idea of the contents.

* [DFA Illustrations (has a Youtube)](http://nbviewer.jupyter.org/github/ganeshutah/Jove/blob/master/notebooks/tutorial/DFAUnit2.ipynb)

* [Regular Operations](http://nbviewer.jupyter.org/github/ganeshutah/Jove/blob/master/notebooks/driver/Drive_AllRegularOps.ipynb)

* [PDA Operations](http://nbviewer.jupyter.org/github/ganeshutah/Jove/blob/master/notebooks/driver/Drive_PDA_Based_Parsing.ipynb)

* [TM Operations](http://nbviewer.jupyter.org/github/ganeshutah/Jove/blob/master/notebooks/driver/Drive_TM.ipynb)

In [ ]:
# This Youtube video walks through this notebook
from IPython.display import YouTubeVideo
YouTubeVideo('L6l3c17mpi4')

# Minimization done two different ways

In [ ]:
RE_Ex3z = "1* 0 1* 0 1* 0 1* + (0* 1 0* (1 0* 1 0*)*)"
NFA_Ex3z = re2nfa(RE_Ex3z)
DO_Ex3z  = dotObj_dfa(min_dfa(nfa2dfa(NFA_Ex3z)))
DO_Ex3z

# The above example, with min_dfa replaced by the rev;det;rev;det

DofNFA_Ex3z = nfa2dfa(re2nfa("1* 0 1* 0 1* 0 1* + (0* 1 0* (1 0* 1 0*)*)"))
dotObj_dfa(DofNFA_Ex3z)
dotObj_dfa(DofNFA_Ex3z)
minDofNFA_Ex3z = nfa2dfa(rev_dfa(nfa2dfa(rev_dfa(DofNFA_Ex3z))))

# Postage Stamp Problem solved using RE and min_dfa

The postage-stamp problem tries to answer the following question (typically posed wrt two postage stamp denominations that are relatively prime)

* Given an unlimited supply of x cent stamps and an unlimited supply of y cent stamps

  - where x and y are relatively prime
  
* Find out the largest denomination that you CANNOT make

* E.g., given 3 cent and 5 cent stamps (x=3, y=5)

  - Check they are relatively prime
  
    * Their GCD must be 1
    
  - Check!
  
* What is the largest postage that CANNOT be made using just 3 and 5 cents?

* Approaches:

  - Trial and error
  
    * 0, 3, 3+3, 3+5, 5+3, 5+5, 5+5+3, ...
    
  - Traditional math books:
  
    * Use Sylvester's theorem to find the Frobenius number
    
  - What I observed about a decade ago (have checked with a few experts and while it is   "out there in the air", nobody had seen it written down...)
  
    * We can use minimal DFA!

    * My approach easily extends to more than two stamps!
    
  - General problem is hard, but has beautiful complexity-theory connections
  
* Onto solving!

## Related Problem: McNugget problem (McNugget Number).
A McNugget number is a positive integer that can be obtained by adding together orders of McDonald's® Chicken McNuggetsTM (prior to consuming any), which originally came in boxes of 6, 9, and 20 (Vardi 1991, pp. 19-20 and 233-234; Wah and Picciotto 1994, p. 186).

__We can find the McNugget Number also using my method !! __

We will do this after solving the postage stamp problem

The numbers are not relatively prime

You will observe something cool when you approach the McNugget problem using REs !!

In [ ]:
# What's the largest postage that can't be made using 3,5 and 7 cents?

# Answer is 4. Find it out.

dotObj_dfa(min_dfa_brz(nfa2dfa(re2nfa("(111+11111+1111111)*"))))